In [18]:
%pip install pandas
%pip install ipywidgets
%pip install matplotlib
%pip install yfinance
%pip install seaborn
%pip install openpyxl
%pip install gradio
%pip install plotly

from ipywidgets import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import seaborn as sns
import gradio as gr
import scipy.optimize as sco
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('dark_background')#this shows graphs in darkbackground
import datetime
from openpyxl import load_workbook
from scipy.stats import gmean
import plotly.graph_objects as go
import plotly.express as px
from scipy.optimize import minimize


#visualisation libraries
from ipywidgets import VBox, HBox
#from views import ReturnsChart, LogScaleReturnsChart

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [12]:
def assess_user(name, age, email, employment, income, net_worth, objective, agree_docs):
    if not agree_docs:
        return "You must agree to the documents to proceed."

    user_data = {
        "Full Name": name,
        "Age": age,
        "Email": email,
        "Employment": employment,
        "Income": income,
        "Net Worth": net_worth,
        "Investment Objective": objective
    }

    summary = f"✅ Thank you, {name}!\n\nHere is your recorded profile:\n"
    for key, value in user_data.items():
        summary += f"- {key}: {value}\n"

    return summary

interface = gr.Interface(
    fn=assess_user,
    inputs=[
        gr.Textbox(label="Full Name"),
        gr.Number(label="Age"),
        gr.Textbox(label="Email"),
        gr.Radio(["Student", "Employed", "Unemployed", "Retired", "Self-employed"], label="Employment Status"),
        gr.Radio(["€0–€30,000", "€30,000–€50,000", "€50,000–€80,000", "€80,000–€120,000", "€120,000+"], label="Annual Income"),
        gr.Radio(["€0–€10,000", "€10,000–€30,000", "€30,000–€100,000", "€100,000+"], label="Net Worth"),
        gr.Dropdown([
            "Long-term investments to maximize wealth growth",
            "Infrequent investments only when I see an opportunity",
            "Frequent investments focused on generating ongoing income",
            "Preserve capital with minimal risk",
            "Invest to learn"
        ], label="Investment Objective"),
        gr.Checkbox(label="I agree to all terms and documents")
    ],
    outputs=gr.Textbox(label="Summary"),
    title="RoboAdvisor - User Questionnaire"
)

interface.launch()

* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.


In [ ]:
def investor_profile_gradio(
    q1, q2, q3, q4, q5, q6, q7, q8, q9, q10
):
    score = 0
    ESG_P = False

    # Question 1
    if "Low risk/low return" in q1: score += 1
    elif "Some risk/medium return" in q1: score += 2
    elif "Higher risk/high return" in q1: score += 3
    elif "Highest risk/highest return" in q1: score += 4

    # Question 2
    if "Change the Portfolio" in q2: score += 1
    elif "Stay the Course" in q2: score += 3

    # Question 3
    if "I agree" in q3: score += 1
    elif "I somewhat agree" in q3: score += 2
    elif "I disagree" in q3: score += 3

    # Question 4
    if "Always" in q4: score += 4
    elif "Usually" in q4: score += 3
    elif "Sometimes" in q4: score += 2
    elif "Never" in q4: score += 1

    # Question 5
    if "strongly agree" in q5: score += 1
    elif "in between" in q5: score += 2
    elif "strongly disagree" in q5: score += 3

    # Question 6
    if "do not plan" in q6: score += 1
    elif "as I am able" in q6: score += 2
    elif "annual contributions" in q6: score += 3

    # Question 7
    if "All of it" in q7: score += 5
    elif "More than half" in q7: score += 4
    elif "Half" in q7: score += 3
    elif "Less than half" in q7: score += 2
    elif "Very little" in q7: score += 1

    # Question 8
    if "$200 gain" in q8: score += 1
    elif "$800 gain" in q8: score += 2
    elif "$2,600 gain" in q8: score += 3
    elif "$4,800 gain" in q8: score += 4

    # Question 9 – ESG
    ESG_P = q9 == "Yes"

    # Classification
    if score <= 12:
        profile = "🟦 Conservative"
        description = "You prioritize protecting your capital over high returns."
    elif score <= 24:
        profile = "🟨 Moderate"
        description = "You balance return and risk with moderate tolerance."
    else:
        profile = "🟥 Aggressive"
        description = "You pursue high growth and accept strong market volatility."

    return f"""**🎯 Profile:** {profile}  
**📊 Risk Score:** {score}  
**🌱 ESG Preference:** {"Yes" if ESG_P else "No"}  
**📈 Experience Investing:** {q10}

**🧭 Summary:** {description}
"""

gr.Interface(
    fn=investor_profile_gradio,
    inputs=[
        gr.Radio([
            "A. Low risk/low return (2–3% yearly; lose 2–5% in a bad year)",
            "B. Some risk/medium return (4% yearly; lose 10%)",
            "C. Higher risk/high return (6% yearly; lose 20%)",
            "D. Highest risk/highest return (7% yearly; lose 25%)"
        ], label="1. If you had €22,000 to invest for 10 years, which would you prefer?"),

        gr.Radio([
            "A. Change the Portfolio",
            "B. Stay the Course"
        ], label="2. If your portfolio drops by 15% in a year, what do you do?"),

        gr.Radio([
            "A. I agree",
            "B. I somewhat agree",
            "C. I disagree"
        ], label="3. When the market drops, do you shift to safer assets?"),

        gr.Radio([
            "A. Always",
            "B. Usually",
            "C. Sometimes",
            "D. Never"
        ], label="4. I would go for the best return even with risk involved."),

        gr.Radio([
            "A. I strongly agree with this statement",
            "B. I’m in between",
            "C. I strongly disagree with this statement"
        ], label="5. I prefer safety and slow growth even if overall return is lower."),

        gr.Radio([
            "A. I do not plan to make future contributions",
            "B. I plan to contribute as I can, not every year",
            "C. I plan to make annual contributions"
        ], label="6. Future investment contributions?"),

        gr.Radio([
            "A. All of it",
            "B. More than half",
            "C. Half",
            "D. Less than half",
            "E. Very little, if any"
        ], label="7. How much would you invest in high-risk opportunities?"),

        gr.Radio([
            "A. $200 gain best / $0 worst",
            "B. $800 gain best / $200 loss",
            "C. $2,600 gain best / $800 loss",
            "D. $4,800 gain best / $2,400 loss"
        ], label="8. Choose a best/worst case investment outcome:"),

        gr.Radio(["Yes", "No", "I'm not sure"], label="9. Would you prioritize ESG even with slightly lower returns?"),

        gr.Radio(["Yes", "No"], label="10. Do you have experience investing in financial markets?")
    ],
    outputs=gr.Markdown(),
    title="💼 Investor Risk Profile + ESG Preferences",
    description="This interactive quiz helps determine your risk appetite, ESG preference, and investment profile."
).launch()


* Running on local URL:  http://127.0.0.1:7867
* To create a public link, set `share=True` in `launch()`.


In [14]:
def fetch_asset_data(start='2019-01-01', end='2024-11-30', save=False):
    asset_tickers = {
        'Bonds': {
            'Bonds US: Treasury Yield 20 Years': '^TYX',
            'Bonds US - High Yield Corporate': 'HYG',
            'Bonds Emerging Markets': 'EMB',
            'Bonds Inflation-Linked: iShares TIPS ETF': 'TIP',
        },
        'Equities': {
            'Equity - US: SP500': 'SPY',
            'Equity - US: Technology Sector': 'XLK',
            'Equity - World: Emerging Markets': 'IEMG',
            'Equity - World: Developed Markets': 'VEA',
        },
        'Real Estate': {
            'Real Estate - Global REIT': 'REET',
            'Real Estate - US REIT ETF': 'VNQ',
            'Real Estate - High Dividend REIT': 'KBWY',
        },
        'Commodities': {
            'Commodities - Gold': 'GC=F',
            'Commodities - Oil': 'CL=F',
            'Commodities - Natural Gas': 'NG=F',
        },
        'Cryptocurrency': {
            'Cryptocurrency - Bitcoin': 'BTC-USD',
            'Cryptocurrency - Ethereum': 'ETH-USD',
        },
        'ESG Funds': {
            'ESG - iShares ESG Aware MSCI USA ETF': 'ESGU',
            'ESG - Vanguard ESG International Stock ETF': 'VSGX',
            'Clean Energy - iShares Global Clean Energy ETF': 'ICLN',
            'Green Bonds - VanEck Green Bond ETF': 'GRNB',
        },
    }

    prices = pd.DataFrame()

    for category, tickers in asset_tickers.items():
        for label, symbol in tickers.items():
            try:
                print(f"Fetching {label} ({symbol})...")
                data = yf.download(symbol, start=start, end=end)
                if 'Adj Close' in data:
                    prices[label] = data['Adj Close']
                else:
                    print(f"Warning: No 'Adj Close' for {symbol}")
            except Exception as e:
                print(f"Error fetching {label}: {e}")

    prices.dropna(inplace=True)
    returns = prices.pct_change().dropna()

    if save:
        prices.to_csv("asset_data.csv")
        returns.to_csv("asset_returns.csv")

    return prices, returns


In [15]:
# Define assets and their tickers
assets = {
    'Bonds US: Treasury Yield 20 Years': '^TYX',
    'Bonds US - High Yield Corporate': 'HYG',
    'Bonds Emerging Markets': 'EMB',
    'Bonds Inflation-Linked: iShares TIPS ETF': 'TIP',
    'Equity - US: SP500': 'SPY',
    'Equity - US: Technology Sector': 'XLK',
    'Equity - World: Emerging Markets': 'IEMG',
    'Equity - World: Developed Markets': 'VEA',
    'Real Estate - Global REIT': 'REET',
    'Real Estate - US REIT ETF': 'VNQ',
    'Real Estate - High Dividend REIT': 'KBWY',
    'Commodities - Gold': 'GLD',  # Changed from GC=F to GLD (ETF proxy)
    'Commodities - Oil': 'USO',   # Changed from CL=F to USO (ETF proxy)
    'Commodities - Natural Gas': 'UNG',
    'Cryptocurrency - Bitcoin': 'BTC-USD',
    'Cryptocurrency - Ethereum': 'ETH-USD',
    'ESG - iShares ESG Aware MSCI USA ETF': 'ESGU',
    'ESG - Vanguard ESG International Stock ETF': 'VSGX',
    'Clean Energy - iShares Global Clean Energy ETF': 'ICLN',
    'Green Bonds - VanEck Green Bond ETF': 'GRNB',
}

df = pd.DataFrame()
failed_assets = []

for name, ticker in assets.items():
    print(f"Fetching {name} ({ticker})...")
    try:
        data = yf.download(ticker, start='2019-01-01', end='2024-11-30', progress=False)
        if 'Adj Close' in data.columns:
            df[name] = data['Adj Close']
        elif 'Close' in data.columns:
            df[name] = data['Close']
        else:
            print(f"⚠️  No usable price column for: {name} ({ticker})")
            failed_assets.append(name)
    except Exception as e:
        print(f"❌ Failed to fetch {name} ({ticker}): {e}")
        failed_assets.append(name)

# Drop rows with missing values
df.dropna(inplace=True)

# Calculate daily returns
df_return = df.pct_change().dropna()

# Save to CSV
df.to_csv("asset_data.csv")
df_return.to_csv("asset_returns.csv")

print("\n✅ Data saved to 'asset_data.csv' and 'asset_returns.csv'")
if failed_assets:
    print("⚠️ The following assets failed to load:")
    for asset in failed_assets:
        print(f"- {asset}")


Fetching Bonds US: Treasury Yield 20 Years (^TYX)...
Fetching Bonds US - High Yield Corporate (HYG)...
Fetching Bonds Emerging Markets (EMB)...
Fetching Bonds Inflation-Linked: iShares TIPS ETF (TIP)...
Fetching Equity - US: SP500 (SPY)...
Fetching Equity - US: Technology Sector (XLK)...
Fetching Equity - World: Emerging Markets (IEMG)...
Fetching Equity - World: Developed Markets (VEA)...
Fetching Real Estate - Global REIT (REET)...
Fetching Real Estate - US REIT ETF (VNQ)...
Fetching Real Estate - High Dividend REIT (KBWY)...
Fetching Commodities - Gold (GLD)...
Fetching Commodities - Oil (USO)...
Fetching Commodities - Natural Gas (UNG)...


/var/folders/22/b18kzt290v58gsf7tgmz2t6h0000gn/T/ipykernel_1802/2070786785.py:31: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start='2019-01-01', end='2024-11-30', progress=False)
/var/folders/22/b18kzt290v58gsf7tgmz2t6h0000gn/T/ipykernel_1802/2070786785.py:31: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start='2019-01-01', end='2024-11-30', progress=False)
/var/folders/22/b18kzt290v58gsf7tgmz2t6h0000gn/T/ipykernel_1802/2070786785.py:31: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start='2019-01-01', end='2024-11-30', progress=False)
/var/folders/22/b18kzt290v58gsf7tgmz2t6h0000gn/T/ipykernel_1802/2070786785.py:31: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start='2019-01-01', end='2024-11-30', progress=False)
/var/folders/22/b18kzt290v58

Fetching Cryptocurrency - Bitcoin (BTC-USD)...
Fetching Cryptocurrency - Ethereum (ETH-USD)...
Fetching ESG - iShares ESG Aware MSCI USA ETF (ESGU)...
Fetching ESG - Vanguard ESG International Stock ETF (VSGX)...
Fetching Clean Energy - iShares Global Clean Energy ETF (ICLN)...
Fetching Green Bonds - VanEck Green Bond ETF (GRNB)...

✅ Data saved to 'asset_data.csv' and 'asset_returns.csv'


/var/folders/22/b18kzt290v58gsf7tgmz2t6h0000gn/T/ipykernel_1802/2070786785.py:31: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start='2019-01-01', end='2024-11-30', progress=False)
/var/folders/22/b18kzt290v58gsf7tgmz2t6h0000gn/T/ipykernel_1802/2070786785.py:31: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start='2019-01-01', end='2024-11-30', progress=False)
/var/folders/22/b18kzt290v58gsf7tgmz2t6h0000gn/T/ipykernel_1802/2070786785.py:31: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start='2019-01-01', end='2024-11-30', progress=False)
/var/folders/22/b18kzt290v58gsf7tgmz2t6h0000gn/T/ipykernel_1802/2070786785.py:31: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start='2019-01-01', end='2024-11-30', progress=False)
/var/folders/22/b18kzt290v58

In [16]:
plt.style.use('dark_background')

# Load pre-saved data
df = pd.read_csv("asset_data.csv", index_col=0, parse_dates=True)
df_return = pd.read_csv("asset_returns.csv", index_col=0, parse_dates=True)

# Portfolio allocations (same as yours)
portfolio_allocations = {
    "Yes": {
        "Conservative": {
            "Bonds US: Treasury Yield 20 Years": 0.25,
            "Bonds US - High Yield Corporate": 0.20,
            "Bonds Emerging Markets": 0.10,
            "Bonds Inflation-Linked: iShares TIPS ETF": 0.15,
            "Commodities - Gold": 0.10,
            "Green Bonds - VanEck Green Bond ETF": 0.20,
        },
        "Moderate": {
            "Equity - US: SP500": 0.15,
            "Equity - US: Technology Sector": 0.15,
            "Equity - World: Developed Markets": 0.10,
            "Bonds US: Treasury Yield 20 Years": 0.10,
            "Bonds US - High Yield Corporate": 0.10,
            "Real Estate - Global REIT": 0.10,
            "Commodities - Gold": 0.10,
            "ESG - iShares ESG Aware MSCI USA ETF": 0.15,
            "Clean Energy - iShares Global Clean Energy ETF": 0.15,
        },
        "Aggressive": {
            "Equity - US: SP500": 0.10,
            "Equity - US: Technology Sector": 0.10,
            "Equity - World: Emerging Markets": 0.10,
            "Real Estate - Global REIT": 0.05,
            "Cryptocurrency - Bitcoin": 0.15,
            "Cryptocurrency - Ethereum": 0.05,
            "Commodities - Gold": 0.05,
            "Commodities - Oil": 0.05,
            "ESG - Vanguard ESG International Stock ETF": 0.10,
            "Clean Energy - iShares Global Clean Energy ETF": 0.15,
        },
    },
    "No": {
        "Conservative": {
            "Bonds US: Treasury Yield 20 Years": 0.30,
            "Bonds US - High Yield Corporate": 0.25,
            "Bonds Emerging Markets": 0.15,
            "Bonds Inflation-Linked: iShares TIPS ETF": 0.20,
            "Commodities - Gold": 0.10,
        },
        "Moderate": {
            "Equity - US: SP500": 0.20,
            "Equity - US: Technology Sector": 0.20,
            "Equity - World: Developed Markets": 0.15,
            "Bonds US: Treasury Yield 20 Years": 0.15,
            "Bonds US - High Yield Corporate": 0.10,
            "Real Estate - Global REIT": 0.10,
            "Commodities - Gold": 0.10,
        },
        "Aggressive": {
            "Equity - US: SP500": 0.15,
            "Equity - US: Technology Sector": 0.15,
            "Equity - World: Emerging Markets": 0.15,
            "Real Estate - Global REIT": 0.10,
            "Cryptocurrency - Bitcoin": 0.20,
            "Cryptocurrency - Ethereum": 0.10,
            "Commodities - Gold": 0.10,
            "Commodities - Oil": 0.05,
        }
    }
}

# Annualized geometric mean return
def ann_geomean(df):
    df = df.dropna()
    years = (df.index[-1] - df.index[0]).days / 365.25
    return ((df + 1).prod()) ** (1 / years) - 1

# Portfolio builder
def get_weighted_portfolio(risk_profile, esg_pref):
    weights_dict = portfolio_allocations[esg_pref][risk_profile]
    tickers = list(weights_dict.keys())
    weights = list(weights_dict.values())

    returns = df_return[tickers].copy().clip(lower=-1, upper=1).fillna(0)
    return returns, tickers, np.array(weights)

# Main logic
def analyze_portfolio(risk_profile, esg_pref):
    portfolio, tickers, weights = get_weighted_portfolio(risk_profile, esg_pref)

    # Weighted cumulative return
    weighted_returns = portfolio.mul(weights, axis=1).sum(axis=1)
    cumulative = (1 + weighted_returns).cumprod()

    # Cumulative return chart
    fig1, ax1 = plt.subplots(figsize=(10, 5))
    cumulative.plot(ax=ax1)
    ax1.set_title(f"Cumulative Portfolio Growth - {risk_profile} | ESG: {esg_pref}")
    ax1.set_ylabel("Growth of $1")
    ax1.set_xlabel("Date")
    ax1.grid(True)
    plt.tight_layout()

    # Metrics
    ann_returns = ann_geomean(portfolio)
    cov_matrix = portfolio.cov() * 252
    portfolio_return = np.dot(ann_returns, weights)
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    sharpe_ratio = (portfolio_return - 0.02) / portfolio_volatility

    # Tables
    asset_metrics = pd.DataFrame({
        "Annual Return": ann_returns,
        "Weight": weights,
        "Contribution to Return": ann_returns * weights
    })

    summary = pd.DataFrame({
        "Portfolio Return": [portfolio_return],
        "Portfolio Volatility": [portfolio_volatility],
        "Sharpe Ratio": [sharpe_ratio]
    })

    # Correlation heatmap
    fig2, ax2 = plt.subplots(figsize=(10, 6))
    sns.heatmap(portfolio.corr(), cmap='vlag', vmin=-1, vmax=1, annot=True, ax=ax2)
    ax2.set_title("Asset Correlation Heatmap")
    plt.tight_layout()

    asset_metrics = asset_metrics.round(3).reset_index()
    asset_metrics.rename(columns={'index': 'Asset'}, inplace=True)

    return fig1, fig2, asset_metrics, summary.round(3)


# Launch Gradio
gr.Interface(
    fn=analyze_portfolio,
    inputs=[
        gr.Radio(["Conservative", "Moderate", "Aggressive"], label="📊 Risk Profile"),
        gr.Radio(["Yes", "No"], label="🌿 ESG Preference")
    ],
    outputs=[
        gr.Plot(label="📈 Portfolio Growth"),
        gr.Plot(label="📉 Correlation Heatmap"),
        gr.Dataframe(label="🧮 Asset-Level Metrics"),
        gr.Dataframe(label="💼 Portfolio Summary")
    ],
    title="Robo-Advisor: Portfolio Insight",
    description="Select your investment style to see how your portfolio might perform."
).launch()


* Running on local URL:  http://127.0.0.1:7868
* To create a public link, set `share=True` in `launch()`.


In [ ]:
def efficient_frontier_plot(esg_option):
    # Load return data
    df_return = pd.read_csv("asset_returns.csv", index_col=0, parse_dates=True)
    df_return = df_return.dropna()

    # Asset filtering
    if esg_option == "ESG":
        selected_assets = [
            "Green Bonds - VanEck Green Bond ETF",
            "ESG - iShares ESG Aware MSCI USA ETF",
            "Clean Energy - iShares Global Clean Energy ETF",
            "ESG - Vanguard ESG International Stock ETF",
            "Equity - US: SP500",
            "Equity - US: Technology Sector",
            "Bonds US - High Yield Corporate",
            "Bonds US: Treasury Yield 20 Years",
            "Commodities - Gold",
            "Real Estate - Global REIT"
        ]
    else:  # Non-ESG
        selected_assets = [
            "Equity - US: SP500",
            "Equity - US: Technology Sector",
            "Equity - World: Emerging Markets",
            "Equity - World: Developed Markets",
            "Cryptocurrency - Bitcoin",
            "Cryptocurrency - Ethereum",
            "Bonds US - High Yield Corporate",
            "Bonds US: Treasury Yield 20 Years",
            "Bonds Inflation-Linked: iShares TIPS ETF",
            "Commodities - Gold",
            "Commodities - Oil",
            "Real Estate - Global REIT"
        ]

    returns = df_return[selected_assets]
    mean_returns = returns.mean() * 252
    cov_matrix = returns.cov() * 252
    risk_free_rate = 0.02

    def portfolio_performance(weights):
        ret = np.dot(weights, mean_returns)
        vol = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
        return ret, vol

    def negative_sharpe_ratio(weights):
        ret, vol = portfolio_performance(weights)
        return -(ret - risk_free_rate) / vol

    def minimize_volatility(weights):
        return portfolio_performance(weights)[1]

    num_assets = len(selected_assets)
    bounds = tuple((0, 1) for _ in range(num_assets))
    constraints = {'type': 'eq', 'fun': lambda x: np.sum(x) - 1}
    initial_weights = num_assets * [1. / num_assets]

    # Optimal portfolios
    opt_sharpe = minimize(negative_sharpe_ratio, initial_weights, method='SLSQP', bounds=bounds, constraints=constraints)
    opt_min_vol = minimize(minimize_volatility, initial_weights, method='SLSQP', bounds=bounds, constraints=constraints)

    # Random portfolios
    n_portfolios = 5000
    results = np.zeros((3, n_portfolios))

    for i in range(n_portfolios):
        weights = np.random.random(num_assets)
        weights /= np.sum(weights)
        r, v = portfolio_performance(weights)
        results[0, i] = v
        results[1, i] = r
        results[2, i] = (r - risk_free_rate) / v

    # Optimal points
    sharpe_return, sharpe_volatility = portfolio_performance(opt_sharpe.x)
    min_return, min_volatility = portfolio_performance(opt_min_vol.x)

    # Plot
    fig, ax = plt.subplots(figsize=(12, 8))
    sc = ax.scatter(results[0, :], results[1, :], c=results[2, :], cmap='coolwarm', alpha=0.5)
    ax.scatter(min_volatility, min_return, marker='*', color='r', s=300, label='Min Volatility')
    ax.scatter(sharpe_volatility, sharpe_return, marker='*', color='g', s=300, label='Max Sharpe Ratio')
    ax.set_title(f'Efficient Frontier - {esg_option} Portfolio')
    ax.set_xlabel('Expected Volatility')
    ax.set_ylabel('Expected Return')
    ax.legend()
    ax.grid(True)
    fig.colorbar(sc, label='Sharpe Ratio')
    plt.tight_layout()

    # Weights Table
    weights_df = pd.DataFrame({
        "Min Volatility Portfolio": opt_min_vol.x,
        "Max Sharpe Ratio Portfolio": opt_sharpe.x
    }, index=returns.columns).applymap(lambda x: f"{x * 100:.2f}%")

    # Return Summary Table
    return_df = pd.DataFrame({
        "Portfolio": ["Min Volatility", "Max Sharpe Ratio"],
        "Expected Annual Return (%)": [f"{min_return * 100:.2f}%", f"{sharpe_return * 100:.2f}%"]
    })

    return fig, weights_df.T.reset_index(), return_df

# Gradio Interface
gr.Interface(
    fn=efficient_frontier_plot,
    inputs=gr.Radio(["ESG", "Non-ESG"], label="🌿 Portfolio Type"),
    outputs=[
        gr.Plot(label="📈 Efficient Frontier"),
        gr.Dataframe(label="💼 Portfolio Weights"),
        gr.Dataframe(label="📊 Expected Returns")
    ],
    title="Efficient Frontier: ESG vs Non-ESG",
    description="Select your portfolio universe and explore optimal portfolios."
).launch()



* Running on local URL:  http://127.0.0.1:7870
* To create a public link, set `share=True` in `launch()`.


/var/folders/22/b18kzt290v58gsf7tgmz2t6h0000gn/T/ipykernel_1802/1580951193.py:95: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  }, index=returns.columns).applymap(lambda x: f"{x * 100:.2f}%")
/var/folders/22/b18kzt290v58gsf7tgmz2t6h0000gn/T/ipykernel_1802/1580951193.py:95: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  }, index=returns.columns).applymap(lambda x: f"{x * 100:.2f}%")


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%pip install cvxpy[glpk]

%pip install ecos
import cvxpy as cp

def efficient_frontier_exact_5_assets(esg_option):
    # Load returns
    df_return = pd.read_csv("asset_returns.csv", index_col=0, parse_dates=True).dropna()

    # Asset selection
    if esg_option == "ESG":
        selected_assets = [
            "Green Bonds - VanEck Green Bond ETF",
            "ESG - iShares ESG Aware MSCI USA ETF",
            "Clean Energy - iShares Global Clean Energy ETF",
            "ESG - Vanguard ESG International Stock ETF",
            "Equity - US: SP500",
            "Equity - US: Technology Sector",
            "Bonds US - High Yield Corporate",
            "Bonds US: Treasury Yield 20 Years",
            "Commodities - Gold",
            "Real Estate - Global REIT"
        ]
    else:
        selected_assets = [
            "Equity - US: SP500",
            "Equity - US: Technology Sector",
            "Equity - World: Emerging Markets",
            "Equity - World: Developed Markets",
            "Cryptocurrency - Bitcoin",
            "Cryptocurrency - Ethereum",
            "Bonds US - High Yield Corporate",
            "Bonds US: Treasury Yield 20 Years",
            "Bonds Inflation-Linked: iShares TIPS ETF",
            "Commodities - Gold",
            "Commodities - Oil",
            "Real Estate - Global REIT"
        ]

    returns = df_return[selected_assets]
    mu = returns.mean() * 252
    cov = returns.cov() * 252
    num_assets = len(selected_assets)
    risk_free = 0.02

    # Variables
    w = cp.Variable(num_assets)
    z = cp.Variable(num_assets, boolean=True)

    # Constraints
    constraints = [
        cp.sum(w) == 1,
        w >= 0.05 * z,         # If selected, weight >= 5%
        w <= z,                # If not selected (z=0), weight = 0
        cp.sum(z) == 5         # Exactly 5 assets
    ]

    # Objective: maximize Sharpe ratio (we linearize as risk-adjusted return)
    expected_return = mu.values @ w
    volatility = cp.quad_form(w, cov.values)
    sharpe_ratio = (expected_return - risk_free) / cp.sqrt(volatility)

    prob = cp.Problem(cp.Maximize(sharpe_ratio), constraints)
    prob.solve(solver=cp.GLPK_MI)

    # Results
    weights = w.value
    selected = np.array(selected_assets)[weights > 1e-4]
    selected_weights = weights[weights > 1e-4]

    # Output Tables
    weights_df = pd.DataFrame({
        "Asset": selected,
        "Weight (%)": [f"{w * 100:.2f}%" for w in selected_weights]
    })

    return_df = pd.DataFrame({
        "Portfolio": ["Max Sharpe Ratio (5 assets)"],
        "Expected Annual Return (%)": [f"{(mu.values @ weights) * 100:.2f}%"],
        "Expected Volatility (%)": [f"{np.sqrt(weights.T @ cov.values @ weights) * 100:.2f}%"],
        "Sharpe Ratio": [((mu.values @ weights - risk_free) / np.sqrt(weights.T @ cov.values @ weights)).round(2)]
    })

    return weights_df, return_df




# Gradio Interface
gr.Interface(
    fn=efficient_frontier_exact_5_assets,
    inputs=gr.Radio(["ESG", "Non-ESG"], label="🌿 Portfolio Type"),
    outputs=[
        gr.Plot(label="📈 Efficient Frontier"),
        gr.Dataframe(label="💼 Portfolio Weights"),
        gr.Dataframe(label="📊 Expected Returns")
    ],
    title="Efficient Frontier: ESG vs Non-ESG",
    description="Select your portfolio universe and explore optimal portfolios."
).launch()

zsh:1: no matches found: cvxpy[glpk]
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
* Running on local URL:  http://127.0.0.1:7882
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/Users/matan-mac/Desktop/Personal/Portfolio/Robo-Advisor/.venv/lib/python3.13/site-packages/gradio/queueing.py", line 626, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<5 lines>...
    )
    ^
  File "/Users/matan-mac/Desktop/Personal/Portfolio/Robo-Advisor/.venv/lib/python3.13/site-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<11 lines>...
    )
    ^
  File "/Users/matan-mac/Desktop/Personal/Portfolio/Robo-Advisor/.venv/lib/python3.13/site-packages/gradio/blocks.py", line 2220, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<8 lines>...
    )
    ^
  File "/Users/matan-mac/Desktop/Personal/Portfolio/Robo-Advisor/.venv/lib/python3.13/site-packages/gradio/blocks.py", line 1731, in call_func

In [38]:
import cvxpy as cp
print(cp.installed_solvers())



['CLARABEL', 'CVXOPT', 'ECOS', 'ECOS_BB', 'GLPK', 'GLPK_MI', 'OSQP', 'SCIPY', 'SCS']


In [37]:
!which python


/Users/matan-mac/Desktop/Personal/Portfolio/Robo-Advisor/.venv/bin/python


In [40]:
import cvxpy as cp
print(cp.installed_solvers())

# Quick test
from cvxpy import Variable, Minimize, Problem

x = Variable()
prob = Problem(Minimize(x), [x >= 1])
prob.solve(solver=cp.GLPK_MI)
print(f"Optimal x = {x.value}")


['CLARABEL', 'CVXOPT', 'ECOS', 'ECOS_BB', 'GLPK', 'GLPK_MI', 'OSQP', 'SCIPY', 'SCS']


SolverError: The solver GLPK_MI is not installed.

In [45]:
import cvxpy as cp
from cvxpy import Variable, Minimize, Problem
import os
os.environ["PATH"] += os.pathsep + "/opt/homebrew/bin"
x = Variable()
prob = Problem(Minimize(x), [x >= 1])
prob.solve(solver=cp.ECOS_BB)
print(f"Optimal x = {x.value}")


SolverError: The solver ECOS_BB is not installed.

In [ ]:
def efficient_frontier_plot(esg_option):
    # Load return data
    df_return = pd.read_csv("asset_returns.csv", index_col=0, parse_dates=True)
    df_return = df_return.dropna()

    # Asset filtering
    if esg_option == "ESG":
        selected_assets = [
            "Green Bonds - VanEck Green Bond ETF",
            "ESG - iShares ESG Aware MSCI USA ETF",
            "Clean Energy - iShares Global Clean Energy ETF",
            "ESG - Vanguard ESG International Stock ETF",
            "Equity - US: SP500",
            "Equity - US: Technology Sector",
            "Bonds US - High Yield Corporate",
            "Bonds US: Treasury Yield 20 Years",
            "Commodities - Gold",
            "Real Estate - Global REIT"
        ]
    else:  # Non-ESG
        selected_assets = [
            "Equity - US: SP500",
            "Equity - US: Technology Sector",
            "Equity - World: Emerging Markets",
            "Equity - World: Developed Markets",
            "Cryptocurrency - Bitcoin",
            "Cryptocurrency - Ethereum",
            "Bonds US - High Yield Corporate",
            "Bonds US: Treasury Yield 20 Years",
            "Bonds Inflation-Linked: iShares TIPS ETF",
            "Commodities - Gold",
            "Commodities - Oil",
            "Real Estate - Global REIT"
        ]

    returns = df_return[selected_assets]
    mean_returns = returns.mean() * 252
    cov_matrix = returns.cov() * 252
    risk_free_rate = 0.02

    def portfolio_performance(weights):
        ret = np.dot(weights, mean_returns)
        vol = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
        return ret, vol

    def negative_sharpe_ratio(weights):
        ret, vol = portfolio_performance(weights)
        return -(ret - risk_free_rate) / vol

    def minimize_volatility(weights):
        return portfolio_performance(weights)[1]

    num_assets = len(selected_assets)
    bounds = tuple((0, 1) for _ in range(num_assets))
    constraints = {'type': 'eq', 'fun': lambda x: np.sum(x) - 1}
    initial_weights = num_assets * [1. / num_assets]

    # Optimal portfolios
    opt_sharpe = minimize(negative_sharpe_ratio, initial_weights, method='SLSQP', bounds=bounds, constraints=constraints)
    opt_min_vol = minimize(minimize_volatility, initial_weights, method='SLSQP', bounds=bounds, constraints=constraints)

    # Random portfolios
    n_portfolios = 5000
    results = np.zeros((3, n_portfolios))

    for i in range(n_portfolios):
        weights = np.random.random(num_assets)
        weights /= np.sum(weights)
        r, v = portfolio_performance(weights)
        results[0, i] = v
        results[1, i] = r
        results[2, i] = (r - risk_free_rate) / v

    # Optimal points
    sharpe_return, sharpe_volatility = portfolio_performance(opt_sharpe.x)
    min_return, min_volatility = portfolio_performance(opt_min_vol.x)

    # Plot
    fig, ax = plt.subplots(figsize=(12, 8))
    sc = ax.scatter(results[0, :], results[1, :], c=results[2, :], cmap='coolwarm', alpha=0.5)
    ax.scatter(min_volatility, min_return, marker='*', color='r', s=300, label='Min Volatility')
    ax.scatter(sharpe_volatility, sharpe_return, marker='*', color='g', s=300, label='Max Sharpe Ratio')
    ax.set_title(f'Efficient Frontier - {esg_option} Portfolio')
    ax.set_xlabel('Expected Volatility')
    ax.set_ylabel('Expected Return')
    ax.legend()
    ax.grid(True)
    fig.colorbar(sc, label='Sharpe Ratio')
    plt.tight_layout()

    # Weights Table
    weights_df = pd.DataFrame({
        "Min Volatility Portfolio": opt_min_vol.x,
        "Max Sharpe Ratio Portfolio": opt_sharpe.x
    }, index=returns.columns).applymap(lambda x: f"{x * 100:.2f}%")

    # Return Summary Table
    return_df = pd.DataFrame({
        "Portfolio": ["Min Volatility", "Max Sharpe Ratio"],
        "Expected Annual Return (%)": [f"{min_return * 100:.2f}%", f"{sharpe_return * 100:.2f}%"]
    })

    return fig, weights_df.T.reset_index(), return_df

# Gradio Interface
gr.Interface(
    fn=efficient_frontier_plot,
    inputs=gr.Radio(["ESG", "Non-ESG"], label="🌿 Portfolio Type"),
    outputs=[
        gr.Plot(label="📈 Efficient Frontier"),
        gr.Dataframe(label="💼 Portfolio Weights"),
        gr.Dataframe(label="📊 Expected Returns")
    ],
    title="Efficient Frontier: ESG vs Non-ESG",
    description="Select your portfolio universe and explore optimal portfolios."
).launch()

* Running on local URL:  http://127.0.0.1:7870
* To create a public link, set `share=True` in `launch()`.


/var/folders/22/b18kzt290v58gsf7tgmz2t6h0000gn/T/ipykernel_1802/1580951193.py:95: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  }, index=returns.columns).applymap(lambda x: f"{x * 100:.2f}%")
/var/folders/22/b18kzt290v58gsf7tgmz2t6h0000gn/T/ipykernel_1802/1580951193.py:95: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  }, index=returns.columns).applymap(lambda x: f"{x * 100:.2f}%")
